<a href="https://colab.research.google.com/github/joselopez11c/mi-primer-proyecto/blob/main/remates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Búsqueda de Remates Judiciales

Este notebook permite buscar, procesar y descargar automáticamente remates judiciales desde el sitio web del Poder Judicial del Perú hasta encontrar un remate específico.

Se debe asignar la variable **NUMERO_REMATE_A_DETENER** con el número del remate donde deseamos que se detenga la búsqueda.

Por ejemplo:

```
NUMERO_REMATE_A_DETENER = '14039'

```

## Instrucciones:

1. Asignar el valor de la variable NUMERO_REMATE_A_DETENER
2. Ejecutar el código dando click en el icono de play.
3. El resultado estará en el Drive, se mostará la ruta al final.


## Observaciones:
- Al ejecutar se solicitará permisos para ejecutar el notebook y conectarse con Drive. *_(Se debe iniciar sesión en una cuenta de Google)_*
- La búsqueda comenzará desde la última publicación en REMAJU.
- La búsqueda terminará cuando se encuentre el número de remate ingresado.
- Al terminar se generará una carpeta en Drive con los PDFs y un archivo XLS del reporte.
- Al terminar la búsqueda se mostrará el resultado en la parte inferior del notebook, las páginas buscadas (de 12 en 12 se hace la búsqueda), y el link de google Drive donde se encuentra la carpeta con los archivos (PDFs y reporte en XLS)

# EJECUTAR CÓDIGO (Antes cambiar el número del remate a detener)

In [17]:
NUMERO_REMATE_A_DETENER = '14039'     # <------ MODIFICAR AQUI Y DAR CLICK EN EJECUTAR

########################################################################################
########################################################################################
################################ JOSELOPEZ11C@GMAIL.COM ################################
########################################################################################
########################################################################################

!pip install pdfplumber
!pip install openpyxl

import re
import os
import requests
import datetime
import pandas as pd
import pdfplumber
from IPython.display import display, HTML
from bs4 import BeautifulSoup
from IPython.display import clear_output
from google.colab import drive


url = 'https://remaju.pj.gob.pe/remaju/pages/publico/remateExterno.xhtml'
urlLogin = 'https://remaju.pj.gob.pe/remaju/pages/seguridad/login.xhtml'

current_date = datetime.datetime.now().strftime('%d_%m_%Y')
save_folder = f'/content/drive/My Drive/remates_{current_date}'
drive.mount('/content/drive')
if not os.path.exists(save_folder):
    os.makedirs(save_folder)


def initialize_session(url_login):
  headers = {
      'Accept': 'application/xml, text/xml, */*; q=0.01',
      'Accept-Language': 'en-US,en;q=0.9,es;q=0.8',
      'Connection': 'keep-alive',
      'Content-Type': 'application/x-www-form-urlencoded',
      'DNT': '1',
      'Faces-Request': 'partial/ajax',
      'Origin': 'https://remaju.pj.gob.pe',
      'Referer': 'https://remaju.pj.gob.pe/remaju/pages/publico/remateExterno.xhtml',
      'Sec-Fetch-Dest': 'empty',
      'Sec-Fetch-Mode': 'cors',
      'Sec-Fetch-Site': 'same-origin',
      'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
      'X-Requested-With': 'XMLHttpRequest',
  }
  response = requests.get(url_login, headers=headers)
  headers['Cookie'] = 'JSESSIONID=' + response.cookies.get_dict().get('JSESSIONID')
  return headers

def fetch_page(headers, first_row):
  dataPagination = {
      'javax.faces.partial.ajax': 'true',
      'javax.faces.source': 'formBuscarRemateExterno:listaRemate',
      'javax.faces.partial.execute': 'formBuscarRemateExterno:listaRemate',
      'javax.faces.partial.render': 'formBuscarRemateExterno:listaRemate',
      'javax.faces.behavior.event': 'page',
      'javax.faces.partial.event': 'page',
      'formBuscarRemateExterno:listaRemate_pagination': 'true',
      'formBuscarRemateExterno:listaRemate_skipChildren': 'true',
      'formBuscarRemateExterno:listaRemate_first': str(first_row),
      'formBuscarRemateExterno:listaRemate_rows': '12',
      'formBuscarRemateExterno': 'formBuscarRemateExterno',
  }
  return requests.post(url, headers=headers, data=dataPagination)

def downloadFile(cookie_header, idFile, view_state_value):
  headers = {
      'Accept': 'application/xml, text/xml, */*; q=0.01',
      'Content-Type': 'application/x-www-form-urlencoded',
      'Cookie': cookie_header
  }

  dataFile = {
      'formBuscarRemateExterno': 'formBuscarRemateExterno',
      idFile: '',
      'javax.faces.ViewState': view_state_value
  }

  return requests.post(url, headers=headers, data=dataFile)


def parse_page_content(page_content):
  soup = BeautifulSoup(page_content, 'xml')
  view_state_value = soup.find('update', {'id': 'j_id1:javax.faces.ViewState:0'}).text
  cdata_content = soup.find('update').text
  return BeautifulSoup(cdata_content, 'html.parser'), view_state_value

def extract_pdf_data(pdf_path):
  text = ""
  with pdfplumber.open(pdf_path) as pdf:
      for page in pdf.pages:
          text += page.extract_text().replace('\n', ' ') if page.extract_text() else ''
  return text

# Buscar todos los div con la clase 'card azul'
def extract_card_data(soup_html, view_state_value, cookie_header):
  cards = soup_html.find_all('div', class_='card azul')
  data = []
  for card in cards:
    # Obtener número de remate y convocatoria
    remate_info = card.find('span', class_='text-bold label-danger h6').get_text(strip=True) if card.find('span', class_='text-bold label-danger h6') else 'No disponible'

    # Usar expresiones regulares para extraer solo el número de remate
    match = re.search(r'Remate N° (\d+)', remate_info)
    if match:
      remate_number = match.group(1)
    else:
      remate_number = 'No disponible'

    # Dividir la información de remate en número y convocatoria, si es necesario
    parts = remate_info.split(" - ") if remate_info != 'No disponible' and len(remate_info.split(" - ")) > 1 else ['No disponible', 'No disponible']
    convocatoria = parts[1] if len(parts) > 1 else 'No disponible'

    # Ubicación, buscando la etiqueta específica y extrayendo el texto siguiente
    location_tag = card.find('i', class_='fa fa-map-marker')
    location = location_tag.find_next_sibling(string=True).strip() if location_tag else 'No disponible'

    # Descripción del inmueble, buscando el contenedor específico
    description = card.find('div', class_='texto-info-scroll').get_text(strip=True) if card.find('div', class_='texto-info-scroll') else 'No disponible'

    # Buscar todos los scripts dentro del card
    scripts = card.find_all('script')
    script_id = scripts[-1].get('id')[:-2] if scripts and scripts[-1].get('id').endswith('_s') else 'Script ID no encontrado'

    # Guardar los datos extraídos en la lista
    data.append({
        'Remate': remate_number,
        'Convocation': convocatoria,
        'Location': location,
        'Description': description,
        'Script ID': script_id,
        'View Code' : view_state_value,
        'Cookie header' : cookie_header
    })
  return data

def update_df_with_pdf_data(df, index, pdf_text, save_folder):
  # Utilizar expresiones regulares para extraer la información del PDF
  demandante_regex = re.search(r"seguidos por (.*?), contra", pdf_text, re.DOTALL)
  demandado_regex = re.search(r", contra (.*?), sobre", pdf_text, re.DOTALL)
  expediente_regex = re.search(r"Expediente Judicial (.*?), tramitado", pdf_text, re.DOTALL)
  juzgado_regex = re.search(r"tramitado ante el (.*?), a cargo", pdf_text, re.DOTALL)

  # Actualizar el DataFrame con los datos extraídos
  df.at[index, 'Demandante'] = demandante_regex.group(1) if demandante_regex else "No encontrado"
  df.at[index, 'Demandado'] = demandado_regex.group(1) if demandado_regex else "No encontrado"
  df.at[index, 'Expediente_Judicial'] = expediente_regex.group(1) if expediente_regex else "No encontrado"
  df.at[index, 'Juzgado'] = juzgado_regex.group(1).replace('\n', ' ').strip() if juzgado_regex else "No encontrado"

  # Adicionalmente, se puede realizar una acción específica si un criterio particular no se cumple
  # Por ejemplo, eliminar el archivo PDF si el demandante no es el esperado
  pdf_file_path = os.path.join(save_folder, f"{df.at[index, 'Remate']}.pdf")
  if "BANCO DE CREDITO DEL PERU" not in (demandante_regex.group(1) if demandante_regex else ""):
      if os.path.exists(pdf_file_path):
          os.remove(pdf_file_path)


def process_pdf_and_update_df(headers, df, save_folder):
  for index, row in df.iterrows():
      file_name = f"Aviso de Remate {row['Remate']}.pdf"
      file_response = downloadFile(headers['Cookie'], row['Script ID'], row['View Code'])
      if file_response.ok and 'application/pdf' in file_response.headers.get('Content-Type', ''):
          pdf_path = os.path.join(save_folder, file_name)
          with open(pdf_path, 'wb') as f:
              f.write(file_response.content)
          pdf_text = extract_pdf_data(pdf_path)
          update_df_with_pdf_data(df, index, pdf_text, save_folder)  # Modified to pass save_folder
      else:
          print("Failed to download a valid PDF file.")
  return df


def process_and_save_data(df, save_folder, current_date):
  # Hacer una copia explícita después de filtrar para evitar SettingWithCopyWarning
  df_filtered = df[df['Demandante'].str.contains("BANCO DE CREDITO DEL PERU", na=False)].copy()

  new_column_names = {
      'Remate': 'Número de Remate',
      'Demandante': 'Demandante',
      'Demandado': 'Demandado',
      'Expediente_Judicial': 'Número de Expediente',
      'Juzgado': 'Juzgado Responsable'
  }

  # Ahora que df_filtered es una copia, puedes cambiar los nombres de las columnas sin problemas
  df_filtered.rename(columns=new_column_names, inplace=True)

  columns_to_include = [
      'Número de Remate',
      'Demandante',
      'Demandado',
      'Número de Expediente',
      'Juzgado Responsable'
  ]

  df_to_save = df_filtered[columns_to_include]

  # Crear la ruta al archivo y guardar el DataFrame en formato Excel
  excel_path = os.path.join(save_folder, f'Informe_REMAJU_{current_date}.xlsx')
  df_to_save.to_excel(excel_path, index=False, engine='openpyxl')
  return df_to_save


def check_for_stop_condition(df, stop_number):
  return any(df['Remate'] == stop_number)


def print_drive_link(save_folder):
  folder_name = save_folder.split('/')[-1]
  link = f"https://drive.google.com/drive/u/0/search?q={folder_name}"
  html_code = f"<div style='text-align: center;'><h1><a href='{link}' target='_blank'>Haga clic aquí para ir al archivo guardado en Google Drive</a></h1></div>"
  display(HTML(html_code))



headers = initialize_session(urlLogin)
df = pd.DataFrame()
stop_found = False
first_row = 0
count = 1
stop_number = NUMERO_REMATE_A_DETENER
clear_output(wait=True)

while not stop_found:
    print(f"Búscando en la página {count}...")
    page_response = fetch_page(headers, first_row)
    soup_html, view_state_value = parse_page_content(page_response.content)
    page_data = extract_card_data(soup_html, view_state_value, headers['Cookie'])
    page_df = pd.DataFrame(page_data)
    df = pd.concat([df, page_df], ignore_index=True)

    # Check if the stop condition is met
    if check_for_stop_condition(page_df, stop_number):
        stop_found = True

    # Preparar para la siguiente página
    first_row += 12
    count += 1

# Continuar con el procesamiento de PDF y guardado de datos solo si se necesita
if stop_found:
    df = process_pdf_and_update_df(headers, df, save_folder)
    df_to_save = process_and_save_data(df, save_folder, current_date)
    print(f"Se encontró el remate N° {stop_number}. Se detiene la búsqueda.")
    print_drive_link(save_folder)  # Display the clickable link
    print()
else:
    print("No se ha encontrado entre las paginas el número: " + stop_number)


df_to_save

Búscando en la página 1
Búscando en la página 2
Búscando en la página 3
Se encontró el remate N° 14039. Se detiene la búsqueda.


,Número de Remate,Demandante,Demandado,Número de Expediente,Juzgado Responsable
0,14066,BANCO DE CREDITO DEL PERU,CORPORACION EDUCATIVA MEZA HERMANOS SOCIEDAD A...,N° 00117-2023-0-0801-JR-CI-01,1° JUZGADO CIVIL - Sede Central DE CAÑETE
3,14063,"CREDICORP CAPITAL SOCIEDAD TITULIZADORA S.A., ...","LUIS ALBERTO LOZANO REYES, INVERSIONES FORESTA...",N° 00778-2011-0-2701-JM-CI-01,JUZGADO CIVIL - SEDE TAMBOPATA DE MADRE DE DIOS
4,14062,BANCO DE CREDITO DEL PERU,JAMES ANTHONY CORTEZ SANTIAGO,N° 00782-2021-0-2001-JR-CI- 03,3° JUZG. CIVIL -Av.F.Chirichigno 351-Urb.El Ch...
21,14045,BANCO DE CREDITO DEL PERU,"JESUS SMITH NEYRA VELASQUEZ, RONAL ALEX SOBRAD...",N° 02141-2018-0-1817-JR-CO-03,3°JUZGADO CIVIL-COMERCIAL DE LIMA
23,14043,BANCO DE CREDITO DEL PERU,CARMEN UCELLI CUZCANO CUETO,N° 14177-2022-0-1817-JR-CO- 03,3°JUZGADO CIVIL-COMERCIAL DE LIMA
27,14039,BANCO DE CREDITO DEL PERU,JOEL JULIO SEGURA HUAMALIAS,N° 06013-2023-0-1817-JR-CO-14,14°JUZGADO CIVIL-COMERCIAL DE LIMA
28,14038,BANCO DE CREDITO DEL PERU,YALMER YVAN VALDIVIEZO ROMERO,N° 08708-2021-0-1817-JR- CO-14,14°JUZGADO CIVIL-COMERCIAL DE LIMA
29,14037,BANCO DE CREDITO DEL PERU,INVERSIONES FLORES FFQ SOCIEDAD ANONIMA CERRAD...,N° 25195-2022-0-1817-JR-CO-14,14°JUZGADO CIVIL-COMERCIAL DE LIMA
34,14032,"BANCO DE CREDITO DEL PERU, JCAP CFG PERU SRL","INVERSIONES NISELY E.I.R.L., YENINA NISELY LOY...",N° 00755-2016-0-0201-JR-CI-01,1°JUZGADO CIVIL SEDE HUARAZ DE ANCASH
